In [7]:
import pandas as pd
from transformers import AutoTokenizer
from datasets import Dataset
import pyarrow as pa
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
from deep_translator import GoogleTranslator
from pattern.text.en import singularize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [4]:
stop_words = set(stopwords.words('english'))
stop_words.remove('who')

def remove_stopwords(text):
    #Translate
    text = GoogleTranslator(source='auto', target='en').translate(text)

    #Lowercase the text
    text = text.lower()

    #Split the text into words
    words = text.split()

    # Perform stemming
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [10]:


tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# Load the dataset from the pandas DataFrame
df = pd.read_csv("data.csv")
df['text'] = df['text'].apply(remove_stopwords)

In [82]:
X = dict(tokenizer(df['text'].to_list(), padding=True, truncation=True, max_length=50, return_tensors='tf'))
y ,unique_label = pd.factorize(df['label'])

In [11]:
class_names = df['label'].unique()
class_names

array(['Full name', 'Age', 'Birthplace', 'Programming Language',
       'General information', 'Coding Project', 'Frameworks',
       'University'], dtype=object)

In [84]:
# Define a TensorFlow model
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import losses

In [86]:
# Train the TensorFlow model on the dataset
model.compile(
    optimizer=optimizers.Adam(learning_rate=5e-5),
    loss=model.hf_compute_loss,
    metrics=['accuracy']
)
model.fit(X, y,epochs=15)

Epoch 1/15
4/4 [==============================] - 31s 2s/step - loss: 2.0532 - accuracy: 0.2019
Epoch 2/15
4/4 [==============================] - 9s 2s/step - loss: 1.8775 - accuracy: 0.3558
Epoch 3/15
4/4 [==============================] - 9s 2s/step - loss: 1.6429 - accuracy: 0.5385
Epoch 4/15
4/4 [==============================] - 12s 3s/step - loss: 1.3888 - accuracy: 0.6154
Epoch 5/15
4/4 [==============================] - 11s 3s/step - loss: 1.1842 - accuracy: 0.7692
Epoch 6/15
4/4 [==============================] - 12s 3s/step - loss: 1.0091 - accuracy: 0.8269
Epoch 7/15
4/4 [==============================] - 11s 3s/step - loss: 0.7873 - accuracy: 0.8750
Epoch 8/15
4/4 [==============================] - 10s 2s/step - loss: 0.6408 - accuracy: 0.9231
Epoch 9/15
4/4 [==============================] - 9s 2s/step - loss: 0.5297 - accuracy: 0.9808
Epoch 10/15
4/4 [==============================] - 9s 2s/step - loss: 0.4190 - accuracy: 0.9904
Epoch 11/15
4/4 [==========================

In [34]:
text = remove_stopwords("How are you")
new_X = dict(tokenizer(text, padding=True, truncation=True, max_length=50, return_tensors='tf'))

predictions = model.predict(new_X,verbose=0)
class_names[tf.argmax(predictions['logits'][0].tolist())]

'General information'

In [95]:
saved_directory = 'model'

tokenizer.save_pretrained(saved_directory)
model.save_pretrained(saved_directory)